In [1]:
import cv2
import torch
from ultralytics import YOLO

In [2]:
def detect_coke_bottle(image):
    """Detects Coca-Cola bottles and returns their bounding boxes and pixel heights."""
    results = coke_model(image)
    coke_boxes = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            height_pixels = y2 - y1  # Calculate height in pixels
            coke_boxes.append((x1, y1, x2, y2, height_pixels))

    return coke_boxes

In [3]:
def detect_height_bottle(image_path, model):
    """Detect Coca-Cola bottles and return their bounding box height in pixels."""
    image = cv2.imread(image_path)
    results = model(image)
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            height_pixels = y2 - y1  # Bottle height in pixels
            real_height_cm = 24  # Known height of Coca-Cola bottle
            
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            return height_pixels, estimated_real_height

    return None, None  # If no bottle is detected

In [4]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

In [5]:
def load_model(model_path):
    model = YOLO(model_path)  # Load YOLO model
    return model

In [6]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image


In [7]:
def get_floor_coordinates(results):
    """Extract the top Y-coordinate and bounding box of the floor"""
    floor_y_top = None
    floor_box = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
                floor_box = (x_min, y_min, x_max, y_max)
    
    return floor_y_top, floor_box

In [8]:
def get_propeller_coordinates(results):
    """Extract the bottom Y-coordinates and bounding boxes of propellers"""
    propeller_boxes = []  # Store bounding boxes of detected propellers
    propeller_y_bottoms = []  # Store Y-bottom of propellers
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
                propeller_boxes.append((x_min, y_min, x_max, y_max))
    
    return propeller_y_bottoms, propeller_boxes

In [9]:
def draw_detections(image, propeller_boxes, floor_box, calibration_obj):
    for i, (x1, y1, x2, y2) in enumerate(propeller_boxes):  # Unpacking tuple directly
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Ensure integer values
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Calculate height from floor to propeller
        pixel_height = y1 - floor_box[1]
        real_height = calculate_height(y1, floor_box[1], calibration_obj)
        height_text = f"Height: {real_height:.2f} cm"
        
        # Draw text label
        cv2.putText(image, height_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Label the calibration object
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        cal_text = f"Calibration: {real_height:.2f} cm ({obj_pixel_height} px)"
        cv2.putText(image, cal_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    return image


In [10]:
def calculate_height(propeller_y, floor_y, calibration_obj=None):
    """
    Calculates the real-world height from the floor to each propeller.
    Uses a calibration object if available.
    """
    if propeller_y is None or floor_y is None:
        print("Propeller or Floor not detected!")
        return None
    
    pixel_height = propeller_y - floor_y  # Distance in pixels
    
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        real_propeller_height = pixel_height * pixel_to_cm_ratio
        return real_propeller_height
    
    return pixel_height  # Returns pixels if no calibration object

In [11]:
import os
image_folder = "Input_test_folder(3m approx)"
output_folder = "Output_test_folder_propeller"
model_floor_path = "best (5).pt"
model_propeller_path = "best (9).pt"
coke_model_path = "best (8).pt"
coke_model = YOLO(coke_model_path)

os.makedirs(output_folder, exist_ok=True)
floor_model = load_model(model_floor_path)
propeller_model = load_model(model_propeller_path)
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    
    
    height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)

    if height_pixels is None:
        print(f"Skipping {image_name}: No Coca-Cola bottle detected.")
        continue

    print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
    print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")
    
    calibration_obj = (estimated_real_height,height_pixels)
    
    floor_results, image = detect_objects(floor_model, image_path)
    floor_y_top, floor_box = get_floor_coordinates(floor_results)
    propeller_results, _ = detect_objects(propeller_model, image_path)
    propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)
    if not propeller_boxes or floor_y_top is None:
        print(f"No propellers or floor detected in {image_path}")
        continue
    print(f"\nImage: {image_path}")
    for i, propeller_y_bottom in enumerate(propeller_y_bottoms):
        real_height = calculate_height(propeller_y_bottom, floor_y_top, calibration_obj)
        print(f"  Propeller {i+1}: {real_height:.2f} cm from floor")
    output_image = draw_detections(image, propeller_boxes, floor_box, calibration_obj)
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, output_image)
    print(f"Output saved at: {output_path}")


0: 800x608 1 750ml coke, 374.1ms
Speed: 4.4ms preprocess, 374.1ms inference, 0.8ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 79.81 pixels
Estimated Real-World Height: 24.00 cm

0: 640x480 1 ceiling, 2 floors, 4 wallss, 345.3ms
Speed: 2.4ms preprocess, 345.3ms inference, 9.4ms postprocess per image at shape (1, 3, 640, 480)

0: 800x608 (no detections), 360.0ms
Speed: 7.4ms preprocess, 360.0ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 608)
No propellers or floor detected in Input_test_folder(3m approx)/image44.jpeg

0: 800x608 1 750ml coke, 348.2ms
Speed: 4.3ms preprocess, 348.2ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 79.93 pixels
Estimated Real-World Height: 24.00 cm

0: 640x480 1 ceiling, 1 floor, 2 wallss, 318.7ms
Speed: 2.9ms preprocess, 318.7ms inference, 6.1ms postprocess per image at shape (1, 3, 640, 480)

0: 800x608 (no detections), 371.1ms
Speed: 4.5ms preproc